# I. Finger counting using convexity defects

In [5]:
import cv2
import numpy as np

#1. Base class for video frame processing
class FrameProcessor:
    def processing_loop(self, source, lth, hth, max_frame_num=-1, alternative_source=""):
        results = []
        frame_count = 0

        # Process each frame
        while True:
            retval, frame = source.read()
            if not retval:
                if alternative_source:
                    source = cv2.VideoCapture(alternative_source)  # Try loading alternative source
                    continue
                break

            result = self.process_frame(frame, lth, hth)
            results.append(result)

            frame_count += 1
            if 0 < max_frame_num <= frame_count:  # Stop if frame limit is reached
                break

            if cv2.waitKey(30) & 0xFF == ord('q'):
                break

        return results

    def process_frame(self, frame, lth, hth):
        return 0  # Base method to process a frame - do nothing
        

#2. Class for real-time HSV color filter tuning
class ColorFilterTuning(FrameProcessor):
    def __init__(self):
        super().__init__()
        cv2.namedWindow("color_filter_parameters")

        cv2.createTrackbar('H low', 'color_filter_parameters', 0, 255, lambda x: None)
        cv2.createTrackbar('S low', 'color_filter_parameters', 0, 255, lambda x: None)
        cv2.createTrackbar('V low', 'color_filter_parameters', 0, 255, lambda x: None)
        cv2.createTrackbar('H high', 'color_filter_parameters', 255, 255, lambda x: None)
        cv2.createTrackbar('S high', 'color_filter_parameters', 255, 255, lambda x: None)
        cv2.createTrackbar('V high', 'color_filter_parameters', 255, 255, lambda x: None)

    def process_frame(self, frame, lth, hth):
        """
        Applies color filtering using dynamically adjusted HSV thresholds.
        """
        
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Get HSV values from trackbars
        lth = (cv2.getTrackbarPos('H low', 'color_filter_parameters'),
               cv2.getTrackbarPos('S low', 'color_filter_parameters'),
               cv2.getTrackbarPos('V low', 'color_filter_parameters'))
        hth = (cv2.getTrackbarPos('H high', 'color_filter_parameters'),
               cv2.getTrackbarPos('S high', 'color_filter_parameters'),
               cv2.getTrackbarPos('V high', 'color_filter_parameters'))

        mask = cv2.inRange(hsv, lth, hth)  # Create binary mask based on HSV range
        cv2.imshow("Frame", frame)
        cv2.imshow("Mask", mask)

        return lth, hth   # Return the tuned HSV values


#3. Class for counting fingers in a frame
class FingersCounter(FrameProcessor):
    """
    Detects and counts fingers based on convexity defects in the hand contour.
    """
    
    def process_frame(self, frame, lth, hth):
        """
        Identifies and counts the number of fingers in the frame.
        """
        
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, lth, hth)

        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((5, 5), np.uint8))

        # Filter to keep only the largest connected component (presumably the hand)
        mask = self.filter_cc(mask, area_th = -1)

        # Fill any remaining small holes inside the detected hand region
        mask = self.fill_holes(mask)

        # Find external contours
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if len(contours) == 0:
            return 0 # no contours are found, no fingers detected

        contour = max(contours, key=cv2.contourArea) # Find the largest contour
        
        if len(contour) < 3:
            return 0 # the contour is too small (<3 points), ignore it
         
        # Find the convex hull (outer shape) of the contour
        hull = cv2.convexHull(contour, returnPoints=False)
        
        if len(hull) < 4:
            return 0 # the convex hull < 4 points, ignore it

        # Sort the convex hull points for consistent processing
        hull = np.sort(hull, axis=0)
        # Find convexity defects
        defects = cv2.convexityDefects(contour, hull)

        if defects is None:
            return 0

        fingers_count = 0

        # Loop through each defect (each valley between fingers)
        for i in range(defects.shape[0]):
            s, e, f, d = defects[i, 0]
            start = tuple(contour[s][0])  # Start point(finger tip)
            end = tuple(contour[e][0])    # End point(another finger tip)
            far = tuple(contour[f][0])    # Deepest point (valley between fingers)

            # Calculate lengths of the triangle sides (finger geometry)
            a = np.linalg.norm(np.array(start) - np.array(far))
            b = np.linalg.norm(np.array(end) - np.array(far))
            c = np.linalg.norm(np.array(start) - np.array(end))

            # calculate the angle at the valley
            angle = np.arccos((a**2 + b**2 - c**2) / (2*a*b))

            if angle < np.pi / 2 and d > 20:   # Threshold for valid defects
                fingers_count += 1

        if fingers_count > 0:
            fingers_count += 1  # If the angle is small enough (less than 90 degrees) and the depth is significant (d > 20),

        cv2.imshow("Frame", frame)
        cv2.imshow("Mask", mask)

        return fingers_count

    # Filter to keep only the largest connected component (presumably the hand)    
    def filter_cc(self, mask, area_th = -1):
        connectivity = 4
        output = cv2.connectedComponentsWithStats(mask, connectivity, cv2.CV_32S)
        num_labels = output[0]
        labels = output[1]
        stats = output[2]
        
        if (num_labels < 1):
            return mask
        
        if (area_th == -1):
            max_area = 1
            max_label = 1
            
            for i in range(1, num_labels):
                area = stats[i, cv2.CC_STAT_AREA]
                
                if (area > max_area):
                    max_area = area
                    max_label = i
            
            for i in range(1, len(stats)):
                if (i != max_label):
                    mask[np.where(labels == i)] = 0
                    
        else:
            for i in range(len(stats)):
                area = stats[i, cv2.CC_STAT_AREA]
                if (area < area_th):
                    mask[np.where(labels == i)] = 0

        return mask

    # Fill any remaining small holes inside the detected hand region
    def fill_holes (self, img):
        (h, w) = img.shape

        before_area = img.sum ()

        img_enlarged = np.zeros ((h + 2, w + 2), np.uint8)
        img_enlarged [1:h+1, 1:w+1] = img

        img_enl_not = cv2.bitwise_not (img_enlarged)
        th, im_th = cv2.threshold (img_enl_not, 220, 255, cv2.THRESH_BINARY_INV);

        im_floodfill = im_th.copy()

        h, w = im_th.shape[:2]
        mask = np.zeros((h+2, w+2), np.uint8)

        cv2.floodFill(im_floodfill, mask, (0,0), 255);
        im_floodfill_inv = cv2.bitwise_not(im_floodfill)
        im_out = im_th | im_floodfill_inv

        result = im_out [1:h-1, 1:w-1]

        return result
        
# Step 1: Tune color filter for skin detection
video_file = "fingers.mov"
cam = cv2.VideoCapture(video_file)
tuner = ColorFilterTuning()
lth, hth = tuner.processing_loop(cam, None, None, alternative_source=video_file)[-1]
print("Color filter parameters:", lth, hth)

# Step 2: Count fingers using the tuned filter
cam = cv2.VideoCapture(video_file)
finger_counter = FingersCounter()
fingers_count_list = finger_counter.processing_loop(cam, lth, hth)

# Step 3: Compare results with reference values and calculate accuracy
reference_fingers_num = [
    5, 5, 1, 0, 0, 5, 5, 5, 0, 0, 0, 5, 5, 5, 5, 4, 3, 3,
    3, 3, 3, 3, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2,
    2, 2, 2, 2, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5,
    2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2,
    2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1,
    3, 4, 0, 0, 0, 1
]
corr_num = sum(1 for ref, actual in zip(reference_fingers_num, fingers_count_list) if ref == actual)
accuracy = corr_num / len(reference_fingers_num)
grade = min(accuracy * 2, 1) * 100

print(f"Your grade is: {int(grade)} out of 100")
print(f"Correct frames: {corr_num} out of {len(reference_fingers_num)}")

cam.release()
cv2.destroyAllWindows()


Color filter parameters: (0, 49, 111) (88, 147, 255)
Your grade is: 100 out of 100
Correct frames: 51 out of 96


# II. Color-shifting effect
Combine the person (with shifted colors) and the background (normal colors)

In [1]:
import cv2
import numpy as np

cv2.namedWindow("frame")
cv2.createTrackbar("HueShift", "frame", 0, 255, lambda x: None)

cap = cv2.VideoCapture(0)

# Read the first frame - the initial background
ret, background = cap.read()

# Apply Gaussian blur to the background to smooth out noise and small movements
background = cv2.GaussianBlur(background, (21, 21), 0)

def shift_hue(frame, hue_shift):
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Shift the Hue channel by the specified amount 
    hsv[:, :, 0] = (hsv[:, :, 0] + hue_shift) % 180

    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

while True:
    ret, frame = cap.read()
    if not ret:
        break  

    # Read current hue shift value from the trackbar
    hue_shift = cv2.getTrackbarPos("HueShift", "frame")
    blurred_frame = cv2.GaussianBlur(frame, (21, 21), 0)

    # Compute the absolute difference between background and current frame
    diff = cv2.absdiff(background, blurred_frame)

    gray_diff = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)

    # Threshold the grayscale difference — only large differences are kept (moving objects)
    _, mask = cv2.threshold(gray_diff, 10, 255, cv2.THRESH_BINARY)
    mask = cv2.dilate(mask, None, iterations=2)

    # Shift the colors of the whole frame using the chosen hue shift
    recolored_frame = shift_hue(frame, hue_shift)

    # Extract just the "person" part (where mask == 255) from the recolored frame
    person = cv2.bitwise_and(recolored_frame, recolored_frame, mask=mask)

    # Extract the background (where mask == 0) from the original frame
    background_part = cv2.bitwise_and(frame, frame, mask=cv2.bitwise_not(mask))

    # Combine the person (with shifted colors) and the background (normal colors)
    result = cv2.add(person, background_part)

    cv2.imshow("frame", result)

    alpha = 0.9
    background = cv2.addWeighted(background, alpha, blurred_frame, 1 - alpha, 0)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
